In [ ]:
#descargar archivos desde url y almacenarlos en carpeta local
import urllib.request as request
import csv
r = request.urlopen('https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
#para imprimir el archivo
''' 
for line in reader:
    print(line)    
'''

In [ ]:
#otra manera e importar los archgivos desde internet
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv',index_col=0)
df.to_csv('truck_event_text_partition.csv')

In [ ]:
import os

for root, dirs, files in os.walk('/TEMP/'):
    for file in files:
        if file.endswith('.ipynb'):
            print(os.path.join(root, file))


In [ ]:
#prueba de commit en github con el archivo de jupyter lab
filenames = [
    "drivers.csv",
    "timesheet.csv",
    "truck_event_text_partition.csv",
]

In [ ]:
#Descargando datos desde repo github
import pandas as pd

url = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/drivers.csv"
drivers = pd.read_csv(url)
drivers.head()

In [ ]:
import pandas as pd
url2="https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/truck_event_text_partition.csv"
truck_event_text_partition=pd.read_csv(url2,index_col=0) #super importante el index para que no ponga el id del registro
truck_event_text_partition.head()

Preparacion

In [ ]:
import sqlite3
conn=sqlite3.connect(':memory:')## aca se indica el nombre de la db
cur= conn.cursor()

Carga de los datos de los eventos de los conductores

In [ ]:
#
# executescript() permite enviar varios comandos de SQL
# en la misma cadena de texto.
#

conn.executescript(
    """
DROP TABLE IF EXISTS truck_events;

CREATE TABLE truck_events (driverId       INT,
                           truckId        INT,
                           eventTime      STRING,
                           eventType      STRING,
                           longitude      DOUBLE,
                           latitude       DOUBLE,
                           eventKey       STRING,
                           correlationId  STRING,
                           driverName     STRING,
                           routeId        STRING,
                           routeName      STRING,
                           eventDate      STRING);
"""
)
conn.commit()

In [ ]:
#
# La función execute() permite enviar únicamente un comando SQL
# al motor de la base de datos. El siguiente comando es equivalente
# a `SHOW TABLES;` en SQL
#
cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

In [ ]:
#
# La siguiente sentencia devuelve un string que
# representa los comandos sql para crear la tabla
#
cur.execute(
    "SELECT sql FROM sqlite_master WHERE type='table' and name='truck_events';"
).fetchall()

In [ ]:
#
# Se imprime el comando sql equivalente para cada
# tabla existente en la base de datos
#
for a in cur.execute("SELECT sql FROM sqlite_master WHERE type='table';").fetchall():
    print(a[0])

<h4>Carga de datos usando INSERT INTO</h4>
Este método de carga consiste en crear una lista de tupas, donde cada elemento de la tupla es un campo de la tabla.


In [ ]:
with open('truck_event_text_partition.csv','rt') as file:
    truck_event=file.readlines()
len(truck_event)
truck_event=[line.replace('\n','') for line in truck_event]
#truck_event=[line[][1:] for line in truck_event]
truck_event[:3]


In [ ]:
truck_event=[line.replace('\n','') for line in truck_event]
# Separa los campos por comas
truck_event=[line.split(',') for line in truck_event]
#convierte a la fila en una tupla
truck_event=[tuple(line) for line in truck_event]
truck_event[:3]
#descartando la primera que es la cabezera 
truck_event=truck_event[1:]
truck_event[:3]

In [ ]:
## Carga a partir de la lista de tuplas
# contenidas en data
cur.executemany("INSERT INTO truck_events VALUES (?,?,?,?,?,?,?,?,?,?,?)", truck_event)
# Verificación
#
cur.execute("SELECT * FROM truck_events LIMIT 1;").fetchall()

<h3> Consulta de datos   </h3>

SELECT
    what
FROM
    tbl_name
[WHERE where-conditions]
[ORDER BY column_name];

In [ ]:
cur.execute("SELECT *FROM truck_events LIMIT 3;").fetchall()